# Mixed ANOVA

Based on [McDermott et al (2016)](https://www.nature.com/articles/nature18635).

This is an R notebook based on what I used to simulate the data and generate the figures for Experiment 1 on multi-way ANOVA. To run the code, click on each block (a "cell") of code and click the "play" button near the top of the page. There is also a button at the top of the page to run all of the cells (the two right arrows, or "fast-forward icon", at the top).

*Written with the help of Microsoft Copilot and Claude Haiku.*

In [ ]:
# Load necessary libraries
library(tidyverse)
library(afex)
library(emmeans)

# Increase plot size
options(repr.plot.width=12, repr.plot.height=7)

### Generate the data

In [ ]:
# Set seed for reproducibility
#set.seed(42)

# Number of subjects and levels (conditions)
n_subjects <- c(47,50) # Number of US and Tsimane respectively
groups <- c('US', 'Tsimane')
n_groups <- length(groups)
chords <- c('Consonant','Dissonant')
n_chords <- length(chords)

# Mean values and standard deviation for each level
means <- array(c(2.9, 2, 3, 3), dim=c(2,2)) # rows are Chord, columns are Group
std_dev_sbj <- 0.5 # standard deviation across participants
std_dev_wthn <- 0.3 # standard deviation within participants

# Initialize empty data frame to store results
df <- data.frame(
  Subject = numeric(),
  Group = character(),
  Chord = character(),
  Pleasantness = numeric()
)

# Simulate data
for (subject in 1:sum(n_subjects)) {
  sbj_avg <- rnorm(1, mean = 0, sd = std_dev_sbj)
  if (subject>n_subjects[1]) { # identify the appropriate group based on the subject ID
      grp = 2
  } else {
      grp = 1
  }
  for (crd in 1:n_chords) {
    value <- rnorm(1, mean = means[crd,grp], sd = std_dev_wthn)
    value <- value + sbj_avg

    # Add row to data frame
    df <- rbind(df, data.frame(
      Subject = subject,
      Group = groups[grp],
      Chord = chords[crd],
      Pleasantness = value
    ))
  }
}

glimpse(df)

In [ ]:
# Plot the data
ggplot(df, aes(x = Chord, y = Pleasantness, group = Subject)) +
  geom_line(aes(color = Subject)) +
  geom_point() +
  facet_wrap(~ Group) +
  labs(x = "Chord", y = "Pleasantness") +
  theme_grey(base_size=20)

In [ ]:
# Calculate mean and standard error
data_summary <- df %>%
  group_by(Chord, Group) %>%
  summarize(
    mean = mean(Pleasantness),
    se = sd(Pleasantness) / sqrt(n())
  )

# Use a line plot with error bars to show the averages
ggplot(data_summary, aes(x = Chord, y = mean, group = Group)) +
    geom_line(aes(color = Group)) +
    geom_point(aes(color = Group)) +
    geom_errorbar(aes(ymin=mean-se, ymax=mean+se, color=Group), width=0.2) +
    labs(y="Pleasantness") +
    theme_grey(base_size=20)

### Run the mixed ANOVA

In [ ]:
# Run the mixed ANOVA (also uses aov_car but with a different model formula)
anova_results <- aov_car(Pleasantness ~ Group + Error(Subject/Chord), data=df)
summary_aov <- summary(anova_results)
summary_aov

In [ ]:
anova_results

### Multiple comparisons

In [ ]:
# Multiple comparisons test with emmeans
emmeans_results <- emmeans(anova_results, specs = pairwise ~ Chord|Group, adj='bonf')
print(emmeans_results$contrasts)